In [ ]:
# pip install neo4j yfiles_jupyter_graphs graphdatascience

In [5]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

class PersonaGraphDB:
    def __init__(self):
        load_dotenv()
        uri = os.getenv("NEO4J_URI")
        user = os.getenv("NEO4J_USER")
        password = os.getenv("NEO4J_PASSWORD")
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def clear_database(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")

    def create_user_node(self, user_id):
        query = "MERGE (:User {id: $user_id})"
        with self.driver.session() as session:
            session.run(query, user_id=user_id)

    def create_conversation_node(self, conv_id, user_id):
        query = """
        MERGE (c:Conversation {id: $conv_id})
        WITH c
        MATCH (u:User {id: $user_id})
        MERGE (u)-[:STARTED]->(c)
        """
        with self.driver.session() as session:
            session.run(query, user_id=user_id, conv_id=conv_id)

    def insert_persona_fact(self, user_id, conv_id, triplet, topic):
        s, r, o = triplet
        query = """
        // Ensure subject and object nodes
        MERGE (sub:Subject {name: $s})
        MERGE (obj:Object {name: $o})
        
        // Create relation edge
        MERGE (sub)-[rel:RELATION {type: $r}]->(obj)

        // Ensure topic and conversation
        MERGE (t:Topic {name: $topic})
        MERGE (c:Conversation {id: $conv_id})
        MERGE (c)-[:MENTIONED]->(t)

        // Connect topic to subject and object
        MERGE (t)-[:HAS]->(sub)
        MERGE (t)-[:HAS]->(obj)
        """
        with self.driver.session() as session:
            session.run(query, s=s, r=r, o=o, topic=topic, conv_id=conv_id, user_id=user_id)


In [6]:
from uuid import uuid4
# from persona_graph_db import PersonaGraphDB  # Assume you saved the class as persona_graph_db.py

# 1. Initialize and clear DB
db = PersonaGraphDB()
db.clear_database()

# 2. Create a test user and conversation
user_id = "user_001"
conv_id = str(uuid4())  # simulate a unique conversation ID
db.create_user_node(user_id)
db.create_conversation_node(conv_id, user_id)

# 3. Create synthetic persona triplets with topics
test_triplets = [
    ("I", "like_watching", "horror"),          # topic: movies
    ("I", "enjoy", "sci-fi"),                  # topic: movies
    ("I", "own", "golden retriever"),          # topic: pets
    ("I", "prefer", "tea"),                    # topic: preferences
    ("I", "play", "guitar"),                   # topic: hobbies
]

# 4. Insert each into Neo4j
for triplet, topic in zip(test_triplets, ["movies", "movies", "pets", "preferences", "hobbies"]):
    db.insert_persona_fact(user_id, conv_id, triplet, topic)

print("✅ Test insertion completed successfully.")

# 5. Close connection
db.close()


✅ Test insertion completed successfully.
